In [451]:
from urllib.request import urlopen, Request
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [4]:
headers = {
    'authority': 'finviz.com',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'nl-BE,nl-NL;q=0.9,nl;q=0.8,en-US;q=0.7,en;q=0.6'}

In [18]:
'''loop to get full table per ticker'''
news_tables = {}
financial = {}
tickers = ['TDOC','RDS-A', 'EURN']

for ticker in tickers:
    print(f"sending request for {ticker}")
    params = (
    ('t', ticker),
    )
    response = requests.get('https://finviz.com/quote.ashx', headers=headers, params=params) 
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response.content, "html.parser")
    # Find 'news-table' in the Soup and load it into 'news_table'
    news_table = html.find('table', class_='fullview-news-outer').find_all('tr')
    snapshot_table = html.find('table', class_='snapshot-table2')
    # Add the table to our dictionary
    news_tables[ticker] = news_table
    financial[ticker] = snapshot_table

sending request for TDOC
sending request for RDS-A
sending request for EURN


In [196]:
'''Get final dict with news'''
new_dict = {}

# Iterate through the news
for file_name, news_table in news_tables.items():
    # Create new list per iteration
    parsed_news = []
    intermediate_dict = {}
    # Iterate through all tr tags in 'news_table'
    for x in news_table:
        # read the text from each tr tag into text
        # get text from a only
        text = x.a.get_text() 
        # splite text in the td tag into a list 
        date_scrape = x.td.text.split()
        # if the length of 'date_scrape' is 1, load 'time' as the only element

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        # else load 'date' as the 1st element and 'time' as the second    
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        # Extract the ticker from the file name, get the string up to the 1st '_'  
        #ticker = file_name.split('_')[0]
        
        # Append ticker, date, time and headline as a list to the 'parsed_news' list
        parsed_news.append([date, text])
    #intermediate dictionary
    intermediate_dict['news'] = parsed_news
    # Append full list to dict
    new_dict[file_name] = intermediate_dict


In [197]:
'''Get financial dict'''
financial_dict = {}
for ticker, values in financial.items():
    table_dark_row = values.find_all('tr', class_="table-dark-row")
    values_dict = {}
    intermediate_dict = {}
    #table_dark_row = financial['TDOC'].find_all('tr', class_="table-dark-row")
    for i in table_dark_row:
        keys = i.find_all('td', class_="snapshot-td2-cp")
        values = i.find_all('td', class_="snapshot-td2")
        for keys, values in zip(keys, values):
            values_dict[keys.text] =  values.text
    intermediate_dict['fundamentals'] = values_dict
    new_dict[ticker]['fundamentals'] = values_dict


In [486]:
'''Clean data'''
def clean_data(data):
    #filling empty cells with nan value and treating '52W Range' column
    #mymap = {'-': float('nan')}
    range_df = data['52W Range']
    data.drop(columns=['52W Range'], axis=1, inplace=True)
    #data.applymap(lambda s: mymap.get(s) if s in mymap else s)
    
    range_df = range_df.apply(lambda x: x.split(' - '))
    data['52W Range Low'] = range_df.apply(lambda x: x[0])
    data['52W Range High'] = range_df.apply(lambda x: x[1])
    #treating % and Million/Billion values to numeric
    mymap = {'%': 0.01, 'M': 1000000, 'B': 1000000000}
    metrics_tonumeric = ['Market Cap', 'Dividend %', 'Shs Outstand']
    for metric in metrics_tonumeric:
        data.loc[:, metric] = pd.to_numeric(data[metric].str[:-1]) * \
                                    data[metric].str[-1].replace(mymap)
    metrics_tonumeric2 = ['EPS next 5Y', 'Insider Own', 
                         'ROE', 'ROI', 'ROA', 'Profit Margin']
    empty_metric = []
    for metric in metrics_tonumeric2:
        empty_metric.append(f"{metric} %")
    
    for new, old in zip(empty_metric, metrics_tonumeric2):
        data[new] = pd.to_numeric(data[old].str[:-1]) * \
                                    data[old].str[-1].replace(mymap)
    #transforming whole df to numeric values
    #data = data.apply(pd.to_numeric, errors='coerce')
    #adjustments on dataframe to make it more readable
    data['Market Cap(B)'] = data['Market Cap'].apply(lambda x: x/1000000000)
    data['Shs Outstand(M)'] = data['Shs Outstand'].apply(lambda x: x/1000000)
    data.replace('-', np.nan, inplace=True)
    columns = ['Market Cap(B)', 'Price', 'Target Price','P/B', 'P/E', 'Forward P/E', 'P/S', 'PEG', 
                         'P/FCF', 'Debt/Eq', 'EPS (ttm)', 'EPS next 5Y %', 'Dividend %', 
                         'Insider Own %', 'ROE %', 'ROI %', 'ROA %', 'Profit Margin %', 
                         'Shs Outstand(M)', 'RSI (14)', 'Beta', '52W Range Low', '52W Range High']
    return data[columns].astype(float)

In [493]:
data = pd.DataFrame.from_records(new_dict['EURN']['fundamentals'], index=['RDS-A'])
data

,52W High,52W Low,52W Range,ATR,Avg Volume,Beta,Book/sh,Cash/sh,Change,Current Ratio,...,Sales Q/Q,Sales past 5Y,Short Float,Short Ratio,Shortable,Shs Float,Shs Outstand,Target Price,Volatility,Volume
RDS-A,-8.57%,35.63%,7.55 - 11.20,0.29,1.74M,-,11.11,-,0.89%,-,...,-72.80%,-,1.81%,1.88,Yes,180.37M,201.68M,12.05,2.03% 2.35%,"799,825"


In [494]:
data = clean_data(data)
data

,Market Cap(B),Price,Target Price,P/B,P/E,Forward P/E,P/S,PEG,P/FCF,Debt/Eq,...,Insider Own %,ROE %,ROI %,ROA %,Profit Margin %,Shs Outstand(M),RSI (14),Beta,52W Range Low,52W Range High
RDS-A,2.07,10.24,12.05,0.92,12.0,53.89,2.23,NaN,NaN,NaN,...,0.0024,NaN,NaN,NaN,NaN,201.68,50.32,NaN,7.55,11.2


In [495]:
type(data['EPS (ttm)'][0])

numpy.float64

In [496]:
'''
Calculation of Intrinsic value of stocks
'''
class CalculateValue(object):
    def __init__(self, data):
        self.data = data
    '''
     Discounted cash flow (DCF) is a valuation method used to estimate the value 
     of an investment based on its future cash flows. DCF analysis attempts to 
     figure out the value of an investment today, based on projections of 
     how much money it will generate in the future.
     param: years = valuation period to be considered
            rate_discount = rate of return that wants to make
            tv_multiplier = terminal value multiplier 
                           (for high quality businesses recommended: 15 and
                            10 for lower quality businesses)
     return: enterprise_value = EV calculated to achieve the set goal
             dcf_value = Stock price calculated to achieve the set goal         
    '''
    def simple_dcf_valuation(self, years, rate_discount, tv_multiplier):
        #free cash flow
        fcf = self.data['Market Cap(B)']/self.data['P/FCF']
          
        growth_1 = self.data['EPS next 5Y %'] #growth of first 5 years
        growth_2 = self.data['EPS next 5Y %']*0.7 #growth of remaining years
          
        compound_list = []
        pv_list = []
        compound_list.append(fcf)
        pv = fcf/(1+rate_discount)
        pv_list.append(pv)
          
        for year in range(2, years//2 + 1):
            new_fcf = fcf + fcf*growth_1
            fcf = new_fcf
            pv = fcf/(1 + rate_discount)**year
            compound_list.append(fcf)
            pv_list.append(pv)
        
        for year in range(years//2 + 1, years + 1):
            new_fcf = fcf + fcf*growth_2
            fcf = new_fcf
            pv = fcf/(1 + rate_discount)**year
            compound_list.append(fcf)
            pv_list.append(pv)
        
        terminal_value = compound_list[-1]*tv_multiplier
        tv_pv = terminal_value/(1 + rate_discount)**years
        pv_list.append(tv_pv)
        enterprise_value = sum(pv_list)
        dcf_value = enterprise_value/self.data['Shs Outstand(M)']*1000 #*1000 because EV(billion)/Shs Outstd(million)
        return enterprise_value, dcf_value
        
    '''
    'Sticker Price'(Fair Value of a Stock) and 'Margin of Safety' valuation method
     param: years = valuation period to be considered
            rate_return = wanted rate of return out of the investment
            margin_safety = is the risk tolerance of the investment. 
                            i.e) a margin of 0.5 means I'll buy the stock
                                 wen priced at 0.5 or less of their 
                                 intrinsic value(fair value)
     return: fairvalue = sticker price or intrinsic value of the stock
             buyvalue = buying price of stock 
    '''
    def sp_valuation(self, years, rate_return, margin_safety):
        eps = self.data['EPS (ttm)']
        growth = self.data['EPS next 5Y %']
        pe = 2*growth*100
        
        compound_list = []
        compound_list.append(eps) #year 1)
        for year in range(years-1):
            new_eps = eps + eps*growth
            eps = new_eps
            compound_list.append(eps)
        values = []
        value = compound_list[-1]*pe
        values.append(value)
        for year in range(years-1):
            value = value/(1 + rate_return)
            values.append(value)
        
        fairvalue = values[-1]
        buyvalue = fairvalue*margin_safety
        return fairvalue, buyvalue

In [497]:
intrinsic_value = CalculateValue(data)
#'Sticker Price'(Fair Value of a Stock) and 'Margin of Safety' valuation method
fairvalue_1, buyvalue_1 = intrinsic_value.sp_valuation(years=10, rate_return=0.15, margin_safety=0.6666)
fairvalue_2, buyvalue_2 = intrinsic_value.sp_valuation(years=10, rate_return=0.15, margin_safety=0.5)
#Discounted cash flow valuation method
enterprise_value_1, dcf_value_1 = intrinsic_value.simple_dcf_valuation(years=10, rate_discount=0.1, tv_multiplier=10)
enterprise_value_2, dcf_value_2 = intrinsic_value.simple_dcf_valuation(years=10, rate_discount=0.1, tv_multiplier=15)

valuation = pd.DataFrame({'DCF EV(B) (tv=10)': enterprise_value_1, 
                          'DCF Value (tv=10': dcf_value_1,
                          'DCF EV(B) (tv=15)': enterprise_value_2, 
                          'DCF Value (tv=15)': dcf_value_2,
                          'SP Fair Value': fairvalue_1, #same as fairvalue_1, so I'll print just one of them
                          'SP Buy Value (ms=0.66)': buyvalue_1,
                          'SP Buy Value (ms=0.5)': buyvalue_2})
results = pd.concat([valuation, data], axis=1)

In [498]:
results

,DCF EV(B) (tv=10),DCF Value (tv=10,DCF EV(B) (tv=15),DCF Value (tv=15),SP Fair Value,SP Buy Value (ms=0.66),SP Buy Value (ms=0.5),Market Cap(B),Price,Target Price,...,Insider Own %,ROE %,ROI %,ROA %,Profit Margin %,Shs Outstand(M),RSI (14),Beta,52W Range Low,52W Range High
RDS-A,NaN,NaN,NaN,NaN,-1.793845,-1.195777,-0.896923,2.07,10.24,12.05,...,0.0024,NaN,NaN,NaN,NaN,201.68,50.32,NaN,7.55,11.2
